In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import pulp

In [26]:
def bmatrix(a):
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [3]:
C = np.array([ [105, 1267, 1134, 2758, 515, 1350, 1311, 1750, 1565, 1137, 3110, 1365, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

In [4]:
A = np.array([  [     1,       1,       1,       1,       1,       1,       0,       0,       0,       0,       0,       0],
                [     0,       0,       0,       0,       0,       0,       1,       1,       1,       1,       1,       1],
                [   100,      50,      50,      50,     100,     100,      50,     100,     100,     100,     100,     100],
                [   100,      50,      50,      25,      50,     150,      50,     100,     100,      50,      50,      50],
                [     0,       0,       0,       0,       0,      12,       0,       0,      12,       0,       0,       0],
                [     1,       1,       2,       1,       1,       3,       1,       2,       2,       2,       3,       1],
                [     6,      29,      32,      14,      54,      60,       0,       0,      18,      13,      29,       8],
                [     2,       0,      17,      23,       0,       0,      22,       6,      50,     102,      77,      31],
                [     0,       0,       0,       0,       0,      12,      35,      24,      80,       0,     112,      58],
                [     9,       8,       7,      34,      26,      22,       0,       0,      52,       0,      43,       0],
                [   1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5],
                [     1,       1,       1,       1,       1,       1,       1,       1,       1,       1,       1,       1],
                [   0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5],
                [   0.5,       0,       0,       0,       0,       0,       0,       0,       0,       0,       0,       0],
                [   390,     355,     370,     430,     440,       0,     275,       0,     100,     133,     213,     100],
                [   275,       0,     470,     305,       0,       0,    1430,     188,    1015,     955,    1343,    1215],
                [     0,       0,     150,       0,       0,     750,     443,     840,    1683,       0,    1748,    1095],
                [   300,     295,     305,     303,     210,     860,       0,       0,       0,       0,       0,       0],
                [345000,  233000,  216000,  242000,  360000,  650000,  244000,  242000,  310000,  263000,  250000,  285000]])

In [10]:
A = np.concatenate((A, np.zeros(A.shape)), axis=1)

In [12]:
I = np.identity(12)

In [15]:
b = 1
p = -(10**10)

A = np.concatenate((A, np.concatenate((-I, b*I), axis=1)), axis=0)
A = np.concatenate((A, np.concatenate((I, p*I), axis=1)), axis=0)

In [16]:
b = np.array([2395,2260,419190,397560,7000,13075,50593,554800,562400,562400,103056,103056,103056,954,20699716,20929709,21159705,21159705,1445855000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
sig = lambda sigma, NUM_ITER: 1/(1 + np.exp(sigma*(np.linspace(0, NUM_ITER, NUM_ITER)-(NUM_ITER/2))))

alpha_lin = np.linspace(0, 1, 100)
alpha_sig_004 = sig(-0.004, 100)
alpha_sig_06 = sig(-0.06, 100)

In [18]:
def create_var():
    var = np.array([])

    lbx = [0] * 12
    lbx[0] = 1500

    lby = [0] * 12

    for i in range(1, 13):
        var = np.append(var, np.array([pulp.LpVariable("x{}".format(i), lowBound = lbx[i-1], upBound=None, cat='Integer')]))

    for i in range(1, 13):
        var = np.append(var, np.array([pulp.LpVariable("y{}".format(i), lowBound = lby[i-1], upBound=None, cat='Binary')]))
    
    return var

In [19]:
def objective(alpha):
    var = create_var()

    Fs = np.array([])

    for c in C:
        Fs = np.append(Fs, c.dot(var))

    fo = alpha*Fs[0] / 7772082 + (1 - alpha) * (Fs[1] / (11*2) + Fs[2] / (3155*2))

    return (var, Fs, fo)

In [20]:
def print_solution(sol, linearProblem, X):
    saida = "\nStatus = {} \nValue = {}".format(pulp.LpStatus[sol], pulp.value(linearProblem.objective))
    
    for i in range(0, X.shape[0]):
        saida += "\n{} = {}".format(X[i], pulp.value(X[i]))
    
    print(saida)

In [21]:
columns=["iter", "alpha", "status"]
for i in range(1, 13):
    columns.append("X{}".format(i))
for i in range(1, 13):
    columns.append("Y{}".format(i))
columns = columns + ["obj_value", "f1", "f2", "f3"]

In [22]:
solution_lst = []

for i in range(0, len(alpha_lin)):

    alpha = alpha_lin[i]
    d_var, F, f = objective(alpha)
    model = pulp.LpProblem("First_Model", pulp.LpMaximize)

    model += f 

    Ax = A.dot(d_var)

    for i in range(0, Ax.shape[0]):
        model += Ax[i] <= b[i]

    solution = model.solve()

    X = np.array([pulp.value(i) for i in d_var])

    fs = C.dot(X)

    solution_lst.append([i,
                        alpha,
                        str(pulp.LpStatus[solution])] +
                        X.tolist() +
                        [pulp.value(model.objective),
                        fs[0],
                        fs[1],
                        fs[2],
                        ])

#print_solution(sol=solution, linearProblem=model, X=d_var)

In [23]:
dt = pd.DataFrame(solution_lst, columns=columns)

In [24]:
dt

,iter,alpha,status,X1,X2,X3,X4,X5,X6,X7,...,Y7,Y8,Y9,Y10,Y11,Y12,obj_value,f1,f2,f3
0,42,0.000000,Optimal,1908.0,2.0,233.0,1.0,1.0,250.0,1923.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.283172,3857429.0,12.0,4655.0
1,42,0.010101,Optimal,1500.0,1.0,1.0,891.0,1.0,1.0,34.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.280495,7912595.0,12.0,4655.0
2,42,0.020202,Optimal,1500.0,1.0,1.0,891.0,1.0,1.0,34.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.277817,7912595.0,12.0,4655.0
3,42,0.030303,Optimal,1500.0,1.0,1.0,891.0,1.0,1.0,34.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.275139,7912595.0,12.0,4655.0
4,42,0.040404,Optimal,1500.0,1.0,1.0,891.0,1.0,1.0,34.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.272462,7912595.0,12.0,4655.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,42,0.959596,Optimal,1500.0,1.0,1.0,891.0,1.0,1.0,34.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.028790,7912595.0,12.0,4655.0
96,42,0.969697,Optimal,1500.0,1.0,1.0,891.0,1.0,1.0,34.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.026112,7912595.0,12.0,4655.0
97,42,0.979798,Optimal,1500.0,1.0,1.0,891.0,1.0,1.0,34.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.023435,7912595.0,12.0,4655.0
98,42,0.989899,Optimal,1500.0,1.0,1.0,892.0,0.0,1.0,33.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.020986,7917997.0,11.0,4655.0


In [25]:
print(bmatrix(A[14:20, 0:12]))
np.set_printoptions(precision = 3, suppress = True)

NameError: name 'bmatrix' is not defined